In [4]:
import requests
from bs4 import BeautifulSoup as bs
import csv
import pandas as pd
import urllib
import re
pd.set_option('display.float_format', lambda x: '%.2f' % x)
from time import sleep

In [5]:
import locale

top_level_data = pd.read_csv("C:\\Users\\adave\\src\\bbox\\mdata\\thenum\\4-7-20\\top_level_movies.csv")
titles = top_level_data["title"]
links = top_level_data["link"]

In [47]:
def get_movie_details(links,how_many = 100, verbose=False):
    counter = 0
    all_movie_info = []
    for link in links:
        try:
            counter += 1
            movie = {}
            r = requests.get(link)
            soup = bs(r.text)
            
            movie = {
                    "Domestic Box Office": 0, 
                    "International Box Office": 0,
                    "Worldwide Box Office": 0,
                    "Opening Weekend:": 0,
                    "Legs:": 0,
                    "Production Budget:": 0,
                    "Theater counts:": 0,
                    "MPAA Rating:": "",
                    "Running Time:": "",
                    "Franchise:": "",
                    "Genre:" : "",
                    "Production Method:": "",
                    "Creative Type:": "",
                    "Production Companies:": ""
                    }

            all_tables = soup.find_all("table")

            #title and year 
            name_area = soup.find("h1", itemprop="name")
            name_s = name_area.string
            name_year = name_s.strip("()").split('(')
            name = name_year[0]
            year = name_year[1]
            
            #all box office
            stats_table = all_tables[0]
            stats_table_data = stats_table.findAll("td")
            for index, rows in enumerate(stats_table_data):
                label = rows.string
                if label in movie:
                    movie[label] = stats_table_data[index+1].string.replace("$","").replace(",","")

            #opening weekend and budget
            label = None
            metric_table = all_tables[2]
            metric_table_data = metric_table.findAll("td")
            for index, rows in enumerate(metric_table_data):
                label = rows.string.strip().replace(u'\xa0', u' ')
                if label in ["Opening Weekend:", "Production Budget:"]:
                    movie[label] = metric_table_data[index + 1].string.split(" ")[0].replace("$","").replace(",","")
                        
                if label in ["Legs:"]:
                    movie[label] = metric_table_data[index + 1].string.split(" ")[0]
                        
                if label in ["Theater counts:"]:
                    pre_count = metric_table_data[index + 1].string.split(", ")[1]
                    movie[label] = pre_count.split(" w")[0]
            
            #runtime, distributor,mpaa, production method, creative type, production companies, and genre
            details_table = all_tables[4]
            details_table_data = details_table.findAll("td")
            for index, rows in enumerate(details_table_data):
                label = rows.string.strip().replace(u'\xa0', u' ')
                if label in ["Running Time:"]:
                    print ("hedhddhdh")
                    movie[label] = details_table_data[index + 1].string.split(" ")[0]
                    
            #print (movie)
            break
            
            #synopsis
            synopsis_section = soup.find("div", {"id": "summary"})
            synopsis_p = synopsis_section.find("p")
            synopsis_u = (synopsis_p).encode(encoding = "UTF-8")
            synopsis_u = str(synopsis_u)
            synopsis = synopsis_u[5:].split("<")[0].replace("\\xc3\\xa2\\xc2\\x80\\xc2\\x99", "'").replace("\\xc3\\xa2\\xc2\\x80\\xc2\\x94", "--").replace("\\xc3\\xa2\\xc2\\x80\\xc2\\xa6", "...").replace("\\xc3\\xa2\\xc2\\x80\\xc2\\x9c", "'").replace("\\xc3\\xa2\\xc2\\x80\\xc2\\x9d", "'").replace("\\xc3\\x83\\xc2\\xa9", "é").replace("\\n", "").replace("\\", "").strip("\\n")

           

            movie ["name"] = name
            movie ["year"] = year
            movie["runtime"] = runtime
            movie ["distributors"] = distributors
            movie ["mpaa_rating"] = mpaa_rating
            movie ["genre"] = genre
            movie ["legs"] = legs
            movie["average_run"] = average_run
            movie ["opening_weekend"] = opening_weekend
            movie ["domestic_box_office"] = domestic_box_office
            movie ["international_box_office"] = international_box_office
            movie ["worldwide_box_office"] = worldwide_box_office
            movie ["synopsis"] = synopsis
            movie["link"] = link
            movie["production_method"] = production_method
            movie["creative_type"] = creative_type

            all_movie_info.append(movie)
            
            if verbose:
                if runtime:
                    print("Runtime: " + runtime)
                print (name)
                print (year)
                print ("Domestic: " + domestic_box_office)
                print ("International: " + international_box_office)
                print ("Worldwide: " + worldwide_box_office)
                print("Opening weekend: " + opening_weekend)
                print ("Legs: " + legs)
                print ("Budget: " + budget)
                print ("Average run: " + average_run + " weeks")
                print ("Mpaa: " + mpaa_rating)
                print ("Genre: " + genre)
                print ("Production method: " + production_method)
                print ("Creative type: " + creative_type)
                print ("Distributors: " + distributors)
                print("Synopsis: " + synopsis)
                print ("----------------------------------------------------------------------------------------------------------------------------")

            if counter >= how_many:
                break
                
            sleep(0.5)
            
        except Exception as ex:
            print(ex)
            print(details_table_data)

            
        
        #return all_movie_info
all_movies = None
all_movies = get_movie_details(links,1,False)
#all_movies

'NoneType' object has no attribute 'strip'
[<td><b>Domestic Releases:</b></td>, <td>December 18th, 2015 (IMAX) by <a href="/market/distributor/Walt-Disney">Walt Disney</a><br/>December 18th, 2015 (Wide) by <a href="/market/distributor/Walt-Disney">Walt Disney</a></td>, <td><b>International Releases:</b></td>, <td>December 16th, 2015 (Wide) (<a href="/movie/Star-Wars-Ep-VII-The-Force-Awakens/Belgium#tab=summary">Belgium</a>)
<br/>December 16th, 2015 (Wide) (<a href="/movie/Star-Wars-Ep-VII-The-Force-Awakens/Denmark#tab=summary">Denmark</a>)
<br/>December 16th, 2015 (Wide) (<a href="/movie/Star-Wars-Ep-VII-The-Force-Awakens/Egypt#tab=summary">Egypt</a>)
<br/>December 16th, 2015 (Wide) (<a href="/movie/Star-Wars-Ep-VII-The-Force-Awakens/Finland#tab=summary">Finland</a>)
<br/>December 16th, 2015 (Wide) (<a href="/movie/Star-Wars-Ep-VII-The-Force-Awakens/France#tab=summary">France</a>)
<br/><a href="#international">... Show all releases</a>
</td>, <td><b><a href="#tab=video-sales">Video Rel

KeyboardInterrupt: 

In [26]:
ord("Opening Weekend:"[7:8])

32

In [126]:
all_movies.to_csv('C:/Users/adave/src/bbox/mdata/thenum/4-7-20/movie_details.csv', index=False)

AttributeError: 'NoneType' object has no attribute 'to_csv'

In [86]:
cols_to_num = ['legs', 'average_run', 'opening_weekend', 'domestic_box_office', 'international_box_office',
               'worldwide_box_office'
              ]
for c in cols_to_num:
    all_movies[c] = pd.to_numeric(all_movies[c])

In [152]:
cols = ['MPAA Rating','Running Time']
movie_data = {}
for row in details_table_data:
    if val of td1 in cols:
        # movie_data[td1 value] = td2 value

SyntaxError: invalid syntax (<ipython-input-152-58b7f48d972f>, line 4)